In [1]:
import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


## Shell Algorithm

In [2]:
def shell(seq): # N log N ou N^6/5 
  inc = len(seq) // 2
  while inc:
    for i, el in enumerate(seq[inc:], inc):
      while i >= inc and seq[i - inc] > el:
        seq[i] = seq[i - inc]
        i -= inc
      seq[i] = el
    inc = 1 if inc == 2 else inc * 5 // 11

## Frequency Count Method

In [3]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(shell)

  # create a list to Shell with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [4]:
reg = PySRRegressor(
    binary_operators=["+","*","/"],
    unary_operators=["log"]
)

reg.fit(X_reshaped, y)

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


[ Info: Started!


PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          432.54013   
	1         1.499287                                   (x0 * 17.571053)   
	2         0.445004                    ((x0 + -5.1265464) * 20.724184)   
	3   >>>>  0.518667                       ((log(x0) * 4.9050417) * x0)   
	4         0.058237              (x0 * (9.978204 + (x0 * 0.20048577)))   
	5         0.041432          ((log(x0 / 1.8380958) / 0.16966115) * x0)   
	6         0.030309  ((x0 * (12.209729 + (x0 * 0.16373974))) + -28....   
	7         0.001853  ((((log(x0) / 0.26418525) + (0.078667566 * x0)...   
	8         0.000239  ((((log(x0 / 0.9501413) / 0.26857924) + (0.079...   
	9         0.000187  ((((log(log(x0)) * (0.076560155 + (8.260883 / ...   
	10        0.000327  (((((log(x0 / 0.9501413) / 0.26857924) + (0.07...   
	11        0.000002  ((((((log(x0 / 0.9501413) / 0.26857924) + (0.0...   
	12        0.003243  ((((((log(x0) / 0.25302446) + (((x0 + -1.70871...   
	
	           loss  complexity  
	0   87693.81000           1  
	1    4372.25240           3  
	2    1795.47490           5  
	3    1068.87100           6  
	4    1008.40150           7  
	5     967.47520           8  
	6     938.59170           9  
	7     933.38904          12  
	8     932.94275          14  
	9     932.76874          15  
	10    932.46400          16  
	11    932.45950          18  
	12    926.43146          20  
]